In [1]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

d="/data/yanghq/models/YHQ/chatglm-6b"

/data/yanghq/utils/anaconda3/envs/glm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1
NVIDIA GeForce RTX 3090


In [2]:
# 载入Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    d, trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [4]:
model = AutoModel.from_pretrained(
    d, trust_remote_code=True).quantize(8).half().cuda()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [5]:
response, history = model.chat(tokenizer, "我是谁？", history=[])
print(response)

NameError: name 'model' is not defined

In [ ]:
print(model)

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): QuantizedLinear(in_features=4096, out_features=12288, bias=True)
          (dense): QuantizedLinear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): QuantizedLinear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): QuantizedLinear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4096, out_features=130528, bias=False)
)


In [ ]:
print(model.transformer.pre_seq_len)

None


In [2]:
import torch
import numpy as np

In [36]:
%time

# 假设b和c的形状分别是[4, C, H, W]和[6, C, H, W]（C, H, W代表其他维度）
#b = torch.randn(4, 4096, 128, 256)  # 随机生成b
#c = torch.randn(6, 4096, 128, 256)  # 随机生成c
b = torch.arange(2,10,2)  # 随机生成b
c = torch.arange(1,19,2)  # 随机生成c
print(b)
print(c)
# 计算合并后的总长度
total_length = b.size(0) + c.size(0)

# 生成b的插入位置，确保b和c的内部顺序不变
positions_b = torch.randperm(total_length)[:b.size(0)]

# 生成一个足够大的张量来容纳结果
result = torch.empty(total_length, *b.shape[1:], dtype=b.dtype)

# 将b和c的元素放入正确位置
result[positions_b] = b
positions_c = torch.tensor([i for i in range(total_length) if i not in positions_b.tolist()])
result[positions_c] = c

print(result)
# 结果张量result现在是b和c的随机合并，且内部顺序保持不变im

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 21.2 µs
tensor([2, 4, 6, 8])
tensor([ 1,  3,  5,  7,  9, 11, 13, 15, 17])
tensor([ 1,  3,  5,  7,  6,  2,  9, 11, 13, 15,  8,  4, 17])


In [17]:
%%time
past_key   past_value
key_layer  value_layer

total_length = past_key.size(0) + key_layer.size(0)
positions_b = torch.randperm(total_length)[:past_key.size(0)].to(dtype=torch.long)
result = torch.empty(total_length, *b.shape[1:], dtype=past_key.dtype)
positions_c = torch.ones(total_length, dtype=torch.bool)
positions_c[positions_b] = False
result[positions_b] = b
result[positions_c] = c

CPU times: user 3.7 s, sys: 8.98 s, total: 12.7 s
Wall time: 1.15 s


In [114]:
def distribute_insert_tensor(insert_tensor, large_tensor,insert_tensor_2,large_tensor_2, dim=0):
    large_length = large_tensor.size(dim)
    insert_length = insert_tensor.size(dim)
    total_length = large_length + insert_length
    indices_insert = torch.randperm(total_length)[:insert_length]
    indices_insert = indices_insert.sort()[0]
    indices_large = torch.ones(total_length, dtype=torch.bool)
    indices_large[indices_insert] = False
    result_tensor = torch.empty(total_length, dtype=insert_tensor.dtype, device=insert_tensor.device)
    result_tensor[indices_insert] = insert_tensor
    result_tensor[indices_large] = large_tensor
    result_tensor_2 = torch.empty(total_length, dtype=insert_tensor.dtype, device=insert_tensor.device)
    result_tensor_2[indices_insert] = insert_tensor_2
    result_tensor_2[indices_large] = large_tensor_2
    return result_tensor,result_tensor_2
b1 = torch.arange(2,10,2)  # 随机生成b
c1 = torch.arange(1,19,2)  # 随机生成c
new_key_layer = distribute_insert_tensor(b1, c1, dim=0)
print(new_key_layer)

(tensor([ 2,  1,  4,  3,  6,  5,  7,  9, 11, 13, 15, 17,  8]), tensor([ 0,  2,  4, 12]))


In [87]:
b1 = torch.arange(2,10,2)  # 随机生成b
c1 = torch.arange(1,19,2)  # 随机生成c
print(b)
print(c)
b = torch.randn(128, 1024, 16, 64)  # 随机生成b
c = torch.randn(2048, 1024, 16, 64)  # 随机生成c

def dispersed_insert(insert_tensor, large_tensor, dim=0):
    total_length = large_tensor.size(dim) + insert_tensor.size(dim)
    insert_positions = torch.randperm(total_length)[:insert_tensor.size(dim)]
    print('',insert_positions)
    result_tensor = torch.zeros(total_length, dtype=insert_tensor.dtype, device=insert_tensor.device)
    large_index, insert_index = 0, 0
    for i in range(total_length):
        if i in insert_positions:
            result_tensor[i] = insert_tensor[insert_index]
            insert_index += 1
        else:
            result_tensor[i] = large_tensor[large_index]
            large_index += 1
    return result_tensor

# 示例使用
# 假设 past_key 和 key_layer 是预先定义的大型4维张量
# 使用上面定义的函数进行操作


tensor([2, 4, 6, 8])
tensor([ 1,  3,  5,  7,  9, 11, 13, 15, 17])


In [128]:
a=b.shape
a=torch.Size(total_length)+a[1:]
a

TypeError: 'int' object is not iterable

In [126]:
type(total_length)

int